In [ ]:
# 🌟 世界クラスモデル - セットアップ
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# GPU環境チェック
try:
    import torch
    print(f"🔥 PyTorch available: {torch.__version__}")
    print(f"💎 CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"🚀 GPU: {torch.cuda.get_device_name(0)}")
except ImportError:
    print("⚠️ PyTorch not available - using CPU fallback")

# 依存関係のインストール（必要に応じて）
try:
    import mlflow
    import wandb
    print("✅ Experiment tracking libraries available")
except ImportError:
    print("⚠️ Installing experiment tracking libraries...")
    # os.system("pip install mlflow wandb")

# プロジェクトパスの設定
sys.path.append(".")
sys.path.append("src")

print("🎯 World-Class Model Training Environment Ready!")


In [ ]:
# 📊 データ読込み & 高度な特徴量生成
import pandas as pd
import numpy as np
from datetime import datetime
import subprocess
import git

# MLflow実験管理の設定
try:
    import mlflow
    import mlflow.sklearn
    
    # Git commit hash を取得
    try:
        repo = git.Repo(search_parent_directories=True)
        git_commit = repo.head.object.hexsha
        print(f"🔗 Git commit: {git_commit[:8]}")
    except:
        git_commit = "unknown"
    
    # MLflow実験開始
    experiment_name = f"world-class-model-{datetime.now().strftime('%Y%m%d')}"
    mlflow.set_experiment(experiment_name)
    
    with mlflow.start_run() as run:
        mlflow.set_tag("git_commit", git_commit)
        mlflow.set_tag("model_type", "world_class_ensemble")
        print(f"🎯 MLflow run started: {run.info.run_id}")
        run_id = run.info.run_id
    
except ImportError:
    print("⚠️ MLflow not available - skipping experiment tracking")
    run_id = None

# データ読込み
print("📥 Loading data...")
try:
    # 既存のユーティリティを使用
    from src.utils import load_raw
    df_raw = load_raw(symbol="SOL_USDT", timeframe="1d", limit=2000)
    print(f"✅ Raw data loaded: {df_raw.shape}")
except:
    try:
        # フォールバック：CSVファイルから読み込み
        df_raw = pd.read_csv("data/raw/SOL_USDT_1d.csv", index_col=0, parse_dates=True)
        print(f"✅ Data loaded from CSV: {df_raw.shape}")
    except:
        # サンプルデータ作成（デモ用）
        print("⚠️ Creating sample data for demonstration")
        dates = pd.date_range(start='2020-01-01', end='2024-01-01', freq='D')
        np.random.seed(42)
        prices = 100 * np.cumprod(1 + np.random.randn(len(dates)) * 0.02)
        df_raw = pd.DataFrame({
            'Open': prices * (1 + np.random.randn(len(dates)) * 0.005),
            'High': prices * (1 + np.abs(np.random.randn(len(dates))) * 0.01),
            'Low': prices * (1 - np.abs(np.random.randn(len(dates))) * 0.01),
            'Close': prices,
            'Volume': np.random.randint(1000, 10000, len(dates))
        }, index=dates)

print(f"📊 Data period: {df_raw.index[0]} to {df_raw.index[-1]}")
print(f"🔢 Data points: {len(df_raw)}")

# 高度な特徴量生成
print("🔬 Generating advanced features...")
try:
    from src.advanced_features import add_advanced_features
    df_features = add_advanced_features(df_raw)
    print(f"✅ Advanced features generated: {df_features.shape}")
except ImportError:
    print("⚠️ Advanced features module not available - using basic features")
    # 基本的な特徴量のみ
    df_features = df_raw.copy()
    df_features['sma_20'] = df_features['Close'].rolling(20).mean()
    df_features['rsi'] = 100 - (100 / (1 + df_features['Close'].rolling(14).apply(
        lambda x: x[x > x.shift(1)].sum() / x[x < x.shift(1)].sum().abs()
    )))

# 欠損値処理
print("🧹 Cleaning data...")
initial_len = len(df_features)
df_features = df_features.dropna()
print(f"🔄 Dropped {initial_len - len(df_features)} rows with NaN values")

print(f"🎯 Final dataset: {df_features.shape}")
print("✅ Data preparation complete!")

# データ概要の表示
print("\n📋 Data Summary:")
df_features.info()
print(f"\n📊 Feature columns: {len(df_features.columns)}")
print(f"🎯 Total features: {len([col for col in df_features.columns if col not in ['Open', 'High', 'Low', 'Close', 'Volume']])}")


In [ ]:
# 🎯 ターゲット作成 & Purged Walk-Forward CV
from sklearn.model_selection import train_test_split

# ターゲット変数の作成（高度な戦略）
print("🎯 Creating advanced target variables...")

# 複数のターゲット戦略
target_strategies = {
    'price_direction': 3,  # 3日後の価格方向
    'volatility_adjusted': 5,  # ボラティリティ調整済みリターン
    'risk_adjusted': 5,  # リスク調整済みリターン
}

# メイン戦略: リスク調整済みリターン
prediction_days = 5
df_ml = df_features.copy()

# 価格リターンの計算
returns = df_ml['Close'].pct_change(prediction_days).shift(-prediction_days)
volatility = returns.rolling(window=20).std()

# リスク調整済みリターン（Sharpe-like）
risk_adjusted_returns = returns / (volatility + 1e-8)

# 高度なターゲット: 上位/下位パーセンタイルベース
percentile_threshold = 0.3
upper_threshold = risk_adjusted_returns.quantile(1 - percentile_threshold)
lower_threshold = risk_adjusted_returns.quantile(percentile_threshold)

# 3クラス分類（Buy=2, Hold=1, Sell=0）からバイナリ分類へ
df_ml['target_continuous'] = risk_adjusted_returns
df_ml['target_3class'] = 1  # Hold
df_ml.loc[risk_adjusted_returns > upper_threshold, 'target_3class'] = 2  # Buy
df_ml.loc[risk_adjusted_returns < lower_threshold, 'target_3class'] = 0  # Sell

# バイナリ分類用（Buy vs Others）
df_ml['target'] = (df_ml['target_3class'] == 2).astype(int)

# 最後のN日分を削除（予測不可能）
df_ml = df_ml[:-prediction_days]

print(f"✅ Target created with {prediction_days} day prediction horizon")
print(f"📊 Target distribution:")
target_counts = df_ml['target'].value_counts()
print(f"  - Class 0 (Others): {target_counts[0]} ({target_counts[0]/len(df_ml)*100:.1f}%)")
print(f"  - Class 1 (Buy): {target_counts[1]} ({target_counts[1]/len(df_ml)*100:.1f}%)")

# 特徴量の選択
feature_columns = [col for col in df_ml.columns if col not in ['target', 'target_3class', 'target_continuous']]
X = df_ml[feature_columns]
y = df_ml['target']

print(f"\n📊 Final ML dataset:")
print(f"  - Samples: {len(X)}")
print(f"  - Features: {len(feature_columns)}")
print(f"  - Positive class rate: {y.mean():.3f}")

# Purged Walk-Forward CV の設定
print("\n🔄 Setting up Purged Walk-Forward Cross-Validation...")
try:
    from src.purged_cv import PurgedWalkForwardCV, purged_cv_score
    
    # Purged CV パラメータ（データリーク防止）
    purged_cv = PurgedWalkForwardCV(
        n_splits=5,
        test_size=len(X) // 10,  # 10%をテストサイズに
        purge_size=prediction_days,  # 予測期間分をパージ
        embargo_size=2  # 追加のエンバーゴ
    )
    
    print(f"✅ Purged CV configured:")
    print(f"  - Splits: {purged_cv.n_splits}")
    print(f"  - Test size: {purged_cv.test_size}")
    print(f"  - Purge size: {purged_cv.purge_size}")
    print(f"  - Embargo size: {purged_cv.embargo_size}")
    
    # CV分割の確認
    print(f"\n📋 CV Split Overview:")
    for i, (train_idx, test_idx) in enumerate(purged_cv.split(X)):
        print(f"  Fold {i+1}: Train[{train_idx[0]:4d}:{train_idx[-1]:4d}] Test[{test_idx[0]:4d}:{test_idx[-1]:4d}]")
    
    cv_available = True
    
except ImportError:
    print("⚠️ Purged CV not available - using standard train/test split")
    cv_available = False
    
    # 標準的な時系列分割
    split_date = df_ml.index[int(len(df_ml) * 0.8)]
    train_mask = df_ml.index <= split_date
    test_mask = df_ml.index > split_date
    
    X_train = X[train_mask]
    X_test = X[test_mask]
    y_train = y[train_mask]
    y_test = y[test_mask]
    
    print(f"✅ Standard split:")
    print(f"  - Train: {len(X_train)} samples")
    print(f"  - Test: {len(X_test)} samples")

print("🎯 Data preparation for world-class modeling complete!")


In [ ]:
# 🚀 世界クラスモデル学習 & 評価
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report
import optuna
from datetime import datetime
import json

# 拡張メトリクス計算関数
def calculate_extended_metrics(y_true, y_pred, y_pred_proba):
    """世界クラスモデル用の拡張メトリクス計算"""
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, average='weighted', zero_division=0),
        'recall': recall_score(y_true, y_pred, average='weighted', zero_division=0),
        'f1_score': f1_score(y_true, y_pred, average='weighted', zero_division=0),
        'auc': roc_auc_score(y_true, y_pred_proba[:, 1]) if y_pred_proba.shape[1] > 1 else 0.5
    }
    
    # 金融特化メトリクス
    returns = y_pred_proba[:, 1] - 0.5  # 予測確率から期待リターン
    if len(returns) > 1:
        sharpe_like = np.mean(returns) / (np.std(returns) + 1e-8) * np.sqrt(252)
        metrics['sharpe_like'] = sharpe_like
        
        # 最大ドローダウン近似
        cum_returns = np.cumsum(returns)
        running_max = np.maximum.accumulate(cum_returns)
        drawdown = (cum_returns - running_max)
        max_dd = np.min(drawdown)
        metrics['max_drawdown_approx'] = max_dd
        
        # Calmar ratio近似
        annual_return = np.mean(returns) * 252
        calmar = annual_return / (abs(max_dd) + 1e-8)
        metrics['calmar_approx'] = calmar
    
    return metrics

# 世界クラスモデル候補の定義
print("🏆 Initializing World-Class Models...")

# 基本モデル（ベンチマーク）
base_models = {
    'lgb_baseline': lgb.LGBMClassifier(
        n_estimators=300,
        max_depth=8,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        verbose=-1
    ),
    'xgb_baseline': xgb.XGBClassifier(
        n_estimators=300,
        max_depth=8,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        verbosity=0
    ),
    'rf_baseline': RandomForestClassifier(
        n_estimators=200,
        max_depth=12,
        random_state=42,
        n_jobs=-1
    )
}

# 高度なモデル
try:
    from src.advanced_models import get_advanced_models, create_ensemble_model, create_stacked_model
    advanced_models = get_advanced_models()
    print("✅ Advanced models loaded")
except ImportError:
    print("⚠️ Advanced models not available - using base models only")
    advanced_models = {}

# 全モデルの結合
all_models = {**base_models, **advanced_models}

print(f"🎯 Total models to train: {len(all_models)}")

# モデル学習 & 評価
print("\n🔥 Training world-class models...")
model_results = {}
trained_models = {}

for model_name, model in all_models.items():
    print(f"\n{'='*60}")
    print(f"🚀 Training {model_name}...")
    print(f"{'='*60}")
    
    try:
        if cv_available:
            # Purged CV による評価
            print("📊 Purged Cross-Validation...")
            cv_scores = purged_cv_score(model, X, y, cv=purged_cv, scoring='roc_auc', verbose=1)
            
            cv_mean = np.mean(cv_scores)
            cv_std = np.std(cv_scores)
            
            print(f"✅ CV AUC: {cv_mean:.4f} (+/- {cv_std:.4f})")
            
            # 全データでの学習
            print("🎯 Training on full dataset...")
            model.fit(X, y)
            
            # 予測（全データ）
            y_pred = model.predict(X)
            y_pred_proba = model.predict_proba(X)
            
            # メトリクス計算
            metrics = calculate_extended_metrics(y, y_pred, y_pred_proba)
            metrics['cv_auc_mean'] = cv_mean
            metrics['cv_auc_std'] = cv_std
            
        else:
            # 標準的な評価
            print("📊 Standard train/test evaluation...")
            model.fit(X_train, y_train)
            
            # 予測
            y_pred = model.predict(X_test)
            y_pred_proba = model.predict_proba(X_test)
            
            # メトリクス計算
            metrics = calculate_extended_metrics(y_test, y_pred, y_pred_proba)
            metrics['cv_auc_mean'] = metrics['auc']
            metrics['cv_auc_std'] = 0.0
        
        # 結果保存
        model_results[model_name] = metrics
        trained_models[model_name] = model
        
        print(f"✅ {model_name} Results:")
        print(f"   AUC: {metrics['auc']:.4f}")
        print(f"   F1:  {metrics['f1_score']:.4f}")
        print(f"   Sharpe-like: {metrics.get('sharpe_like', 0):.4f}")
        print(f"   Max DD: {metrics.get('max_drawdown_approx', 0):.4f}")
        print(f"   Calmar: {metrics.get('calmar_approx', 0):.4f}")
        
        # MLflowログ
        if run_id:
            try:
                with mlflow.start_run(run_id=run_id):
                    for metric_name, metric_value in metrics.items():
                        mlflow.log_metric(f"{model_name}_{metric_name}", metric_value)
            except:
                pass
        
    except Exception as e:
        print(f"❌ Error training {model_name}: {str(e)}")
        model_results[model_name] = {'error': str(e)}

# 結果サマリー
print(f"\n{'='*80}")
print("🏆 WORLD-CLASS MODEL RESULTS SUMMARY")
print(f"{'='*80}")

# 結果をDataFrameに変換
results_df = pd.DataFrame(model_results).T
valid_results = results_df[~results_df.index.str.contains('error', na=False)]

if len(valid_results) > 0:
    # トップモデルを特定
    valid_results = valid_results.select_dtypes(include=[np.number])
    
    print("\n📊 Model Performance Ranking:")
    print("-" * 50)
    
    # AUCでランキング
    if 'auc' in valid_results.columns:
        top_models = valid_results.sort_values('auc', ascending=False)
        for i, (model_name, row) in enumerate(top_models.head(5).iterrows()):
            print(f"{i+1:2d}. {model_name:20s} - AUC: {row['auc']:.4f} | F1: {row['f1_score']:.4f}")
    
    # 世界クラス基準のチェック
    print(f"\n🎯 World-Class Criteria Check:")
    print("-" * 40)
    
    world_class_candidates = []
    for model_name, metrics in model_results.items():
        if isinstance(metrics, dict) and 'error' not in metrics:
            auc = metrics.get('auc', 0)
            sharpe_like = metrics.get('sharpe_like', 0)
            max_dd = metrics.get('max_drawdown_approx', 0)
            
            world_class_score = 0
            if auc > 0.65: world_class_score += 1
            if sharpe_like > 1.5: world_class_score += 1
            if max_dd > -0.1: world_class_score += 1  # Less than 10% DD
            
            if world_class_score >= 2:
                world_class_candidates.append((model_name, world_class_score, metrics))
                print(f"✅ {model_name}: Score {world_class_score}/3")
    
    # トップ2モデルを選択
    if len(world_class_candidates) >= 2:
        world_class_candidates.sort(key=lambda x: x[1], reverse=True)
        top_2_models = world_class_candidates[:2]
        
        print(f"\n🏆 TOP 2 WORLD-CLASS MODELS SELECTED:")
        for i, (model_name, score, metrics) in enumerate(top_2_models):
            print(f"{i+1}. {model_name} (Score: {score}/3)")
            print(f"   AUC: {metrics['auc']:.4f} | Sharpe: {metrics.get('sharpe_like', 0):.4f}")
        
        # 次のステップ用にトップモデルを保存
        top_2_model_names = [name for name, _, _ in top_2_models]
        print(f"\n🎯 Ready for ensemble/stacking: {top_2_model_names}")
        
    else:
        print("⚠️ No models met world-class criteria. Using best available models.")
        if len(valid_results) > 0:
            top_2_model_names = list(valid_results.sort_values('auc', ascending=False).head(2).index)
        else:
            top_2_model_names = []
    
    print(f"\n✅ Model training complete! {len(valid_results)} models trained successfully.")
    
else:
    print("❌ No models trained successfully.")
    top_2_model_names = []

print(f"\n🎯 Next: Ensemble/Stacking with top models for ultimate performance!")


In [ ]:
# 🏆 メタモデル構築 & 最終保存
import pickle
import os

print("🔮 Creating Ultimate Meta-Model...")

# メタモデルの構築
if len(top_2_model_names) >= 2:
    print(f"🎯 Building ensemble with top models: {top_2_model_names}")
    
    # トップ2モデルを取得
    top_models = [trained_models[name] for name in top_2_model_names]
    
    try:
        # Stacking Model を作成
        from src.advanced_models import create_stacked_model
        meta_model = create_stacked_model(top_models)
        
        print("🔥 Training Stacked Meta-Model...")
        if cv_available:
            meta_model.fit(X, y)
            final_pred = meta_model.predict(X)
            final_pred_proba = meta_model.predict_proba(X)
            final_metrics = calculate_extended_metrics(y, final_pred, final_pred_proba)
        else:
            meta_model.fit(X_train, y_train)
            final_pred = meta_model.predict(X_test)
            final_pred_proba = meta_model.predict_proba(X_test)
            final_metrics = calculate_extended_metrics(y_test, final_pred, final_pred_proba)
        
        final_model = meta_model
        final_model_name = "stacked_meta_model"
        
    except ImportError:
        print("⚠️ Stacking not available - using ensemble averaging")
        
        # アンサンブル平均
        if cv_available:
            all_probas = []
            for model_name in top_2_model_names:
                model = trained_models[model_name]
                proba = model.predict_proba(X)
                all_probas.append(proba[:, 1])
            
            ensemble_proba = np.mean(all_probas, axis=0)
            final_pred = (ensemble_proba > 0.5).astype(int)
            final_pred_proba = np.column_stack([1 - ensemble_proba, ensemble_proba])
            final_metrics = calculate_extended_metrics(y, final_pred, final_pred_proba)
        else:
            all_probas = []
            for model_name in top_2_model_names:
                model = trained_models[model_name]
                proba = model.predict_proba(X_test)
                all_probas.append(proba[:, 1])
            
            ensemble_proba = np.mean(all_probas, axis=0)
            final_pred = (ensemble_proba > 0.5).astype(int)
            final_pred_proba = np.column_stack([1 - ensemble_proba, ensemble_proba])
            final_metrics = calculate_extended_metrics(y_test, final_pred, final_pred_proba)
        
        final_model = trained_models[top_2_model_names[0]]  # 最高性能モデルを代表として保存
        final_model_name = "ensemble_meta_model"

else:
    print("⚠️ Using single best model as final model")
    if len(valid_results) > 0:
        best_model_name = valid_results.sort_values('auc', ascending=False).index[0]
        final_model = trained_models[best_model_name]
        final_model_name = best_model_name
        final_metrics = model_results[best_model_name]
    else:
        print("❌ No valid models available")
        final_model = None
        final_model_name = None
        final_metrics = {}

# 最終結果の表示
if final_model is not None:
    print(f"\n{'='*80}")
    print("🎉 FINAL WORLD-CLASS MODEL RESULTS")
    print(f"{'='*80}")
    print(f"Model: {final_model_name}")
    print(f"AUC: {final_metrics.get('auc', 0):.4f}")
    print(f"F1 Score: {final_metrics.get('f1_score', 0):.4f}")
    print(f"Sharpe-like: {final_metrics.get('sharpe_like', 0):.4f}")
    print(f"Max Drawdown: {final_metrics.get('max_drawdown_approx', 0):.4f}")
    print(f"Calmar Ratio: {final_metrics.get('calmar_approx', 0):.4f}")
    
    # 世界クラス基準の最終チェック
    auc = final_metrics.get('auc', 0)
    sharpe_like = final_metrics.get('sharpe_like', 0)
    max_dd = final_metrics.get('max_drawdown_approx', 0)
    
    print(f"\n🎯 World-Class Criteria Achievement:")
    print(f"AUC > 0.65: {'✅' if auc > 0.65 else '❌'} ({auc:.4f})")
    print(f"Sharpe > 1.5: {'✅' if sharpe_like > 1.5 else '❌'} ({sharpe_like:.4f})")
    print(f"MaxDD > -10%: {'✅' if max_dd > -0.1 else '❌'} ({max_dd:.4f})")
    
    # 目標達成度
    total_score = sum([auc > 0.65, sharpe_like > 1.5, max_dd > -0.1])
    if total_score >= 2:
        print(f"\n🏆 WORLD-CLASS CRITERIA MET! Score: {total_score}/3")
    else:
        print(f"\n⚠️ Criteria partially met. Score: {total_score}/3")

# モデル保存
print(f"\n💾 Saving models...")

today = datetime.now().strftime("%Y%m%d")
model_dir = f"data/models/{today}"
os.makedirs(model_dir, exist_ok=True)

if final_model is not None:
    # メインモデル保存
    model_path = os.path.join(model_dir, "model.pkl")
    with open(model_path, 'wb') as f:
        pickle.dump(final_model, f)
    print(f"✅ Main model saved: {model_path}")
    
    # 特徴量リスト保存
    features_path = os.path.join(model_dir, "features.json")
    with open(features_path, 'w', encoding='utf-8') as f:
        json.dump(feature_columns, f, ensure_ascii=False, indent=2)
    print(f"✅ Features saved: {features_path}")
    
    # メタデータ保存
    metadata = {
        "model_info": {
            "model_type": final_model_name,
            "model_path": model_path,
            "feature_list_path": features_path,
            "creation_date": today,
            "creation_datetime": datetime.now().isoformat(),
            "top_models_used": top_2_model_names if len(top_2_model_names) >= 2 else [final_model_name]
        },
        "data_info": {
            "symbol": "SOL_USDT",
            "timeframe": "1d",
            "prediction_days": prediction_days,
            "total_samples": len(df_ml),
            "feature_count": len(feature_columns),
            "target_distribution": {
                "class_0_count": int(target_counts[0]),
                "class_1_count": int(target_counts[1]),
                "positive_rate": float(y.mean())
            }
        },
        "model_performance": final_metrics,
        "world_class_criteria": {
            "auc_threshold": 0.65,
            "sharpe_threshold": 1.5,
            "max_dd_threshold": -0.1,
            "criteria_met": total_score if final_model else 0,
            "total_criteria": 3
        },
        "training_parameters": {
            "purged_cv_used": cv_available,
            "cv_splits": purged_cv.n_splits if cv_available else None,
            "advanced_features_used": True,
            "ensemble_method": "stacking" if len(top_2_model_names) >= 2 else "single_model"
        },
        "experiment_tracking": {
            "mlflow_run_id": run_id,
            "git_commit": git_commit if 'git_commit' in locals() else "unknown"
        }
    }
    
    metadata_path = os.path.join(model_dir, "metadata.json")
    with open(metadata_path, 'w', encoding='utf-8') as f:
        json.dump(metadata, f, ensure_ascii=False, indent=2)
    print(f"✅ Metadata saved: {metadata_path}")
    
    # すべてのモデル結果を保存
    all_results_path = os.path.join(model_dir, "all_model_results.json")
    with open(all_results_path, 'w', encoding='utf-8') as f:
        # numpy型を通常のPython型に変換
        serializable_results = {}
        for k, v in model_results.items():
            if isinstance(v, dict):
                serializable_results[k] = {
                    key: float(val) if isinstance(val, (np.floating, np.integer)) else val 
                    for key, val in v.items()
                }
            else:
                serializable_results[k] = v
        json.dump(serializable_results, f, ensure_ascii=False, indent=2)
    print(f"✅ All results saved: {all_results_path}")

# 最終サマリー
print(f"\n{'='*80}")
print("🎯 WORLD-CLASS MODEL TRAINING COMPLETE!")
print(f"{'='*80}")
print(f"📂 Models saved in: {model_dir}")
print(f"🎯 Final model: {final_model_name}")
print(f"✅ Next step: Run 03_backtest.ipynb for comprehensive backtesting")

if run_id:
    print(f"📊 MLflow run ID: {run_id}")
    print(f"🔗 View experiments in MLflow UI")

print(f"\n🚀 Ready to achieve Sharpe > 3 and MaxDD < 5% in backtesting!")
print("🏆 World-class model development completed successfully! 🏆")
